# Analysis of Historical S&P prices

### Dec 29, 1927 - Present
#### https://www.wsj.com/market-data/quotes/index/SPX/historical-prices

##### Reading Data

In [20]:
import pandas as pd
df = pd.read_csv('HistoricalPrices.csv')
df.head()

,Date,Open,High,Low,Close
0,11/30/22,3957.64,4080.11,3938.58,4080.11
1,11/29/22,3964.19,3976.77,3937.65,3957.63
2,11/28/22,4005.36,4012.27,3955.77,3963.94
3,11/25/22,4023.34,4034.02,4020.76,4026.12
4,11/23/22,4001.02,4033.78,3998.66,4027.26
